## 0. 导入依赖

In [ ]:
from configparser import ConfigParser          # 读取 config.ini
from pathlib import Path                       # 处理路径
from pickle import dump                        # 保存构建好的 MicroCorpus

import sys
import os
sys.path.append(os.path.abspath(".."))  # 从 notebooks/ 回到外面的 MiCoGPT 根目录
from MiCoGPT.utils.mgm_MicroCorpus import MicroCorpus   # 核心数据集类
from MiCoGPT.utils.mgm_utils import CustomUnpickler     # 自定义 Unpickler，用于反序列化 MicroTokenizer

## 1. 定义 construct_simple 函数

In [ ]:
def construct_simple(
    input_path: str,       # 输入丰度表文件路径（h5 / csv / tsv / txt）
    output_path: str,      # 输出的 MicroCorpus pkl 路径
    config_path: str,      # config.ini 的路径
    tokenizer_path: str,   # MicroTokenizer.pkl 的路径
    key: str = "genus",
    no_normalize: bool = False,  # 是否跳过归一化
):

    # config.ini，拿到 max_len
    cfg = ConfigParser()
    cfg.read(config_path)
    max_len = cfg.getint("construct", "max_len")

    print(f"[construct] 从配置文件中读取到 max_len = {max_len}")


    # 加载 MicroTokenizer.pkl
    tokenizer_path = Path(tokenizer_path)
    if not tokenizer_path.is_file():
        raise FileNotFoundError(f"找不到 tokenizer 文件：{tokenizer_path}")

    with open(tokenizer_path, "rb") as f:
        # 用自定义的 CustomUnpickler 来反序列化，是为了在不同环境下
        # 也能正确找到 MicroTokenizer 类所在的模块。
        unpickler = CustomUnpickler(f)
        tokenizer = unpickler.load()

    # 顺便打印一点信息帮助理解 tokenizer
    try:
        vocab_size = len(tokenizer.vocab)
        print(f"[construct] tokenizer 加载完成，词表大小 = {vocab_size}")
    except Exception:
        print("[construct] tokenizer 已加载（未访问 vocab，可能实现不同）")

    # 打印归一化提示
    if not no_normalize:
        print(
            "Your data will be normalized with the phylogeny mean and std. "
            "If you wish to use your own normalization, please set no_normalize=True."
        )


    #  构建 MicroCorpus
    input_path = Path(input_path)
    if not input_path.is_file():
        raise FileNotFoundError(f"找不到输入丰度表文件：{input_path}")

    corpus = MicroCorpus(
        data_path=str(input_path),
        tokenizer=tokenizer,
        key=key,
        max_len=max_len,
        preprocess=not no_normalize,
    )

    print("[construct] MicroCorpus 已构建完成。")
    try:
        print(f"[construct] 语料库样本数量 = {len(corpus)}")
    except TypeError:
        print("[construct] corpus 不支持 len()，可以自行检查内部实现。")


    # 保存到 output_path
    output_path = Path(output_path)
    with open(output_path, "wb") as f:
        dump(corpus, f)

    print(f"[construct] 语料库已保存到：{output_path.resolve()}")

    # 返回 corpus，方便后续在 Notebook 里分析
    return corpus

## 2. 执行

In [3]:
# 设置根目录
project_root = Path("/Users/kiancai/STA24/CWD/STAi/MiCoGPT")

# 生成路径
input_path = project_root / "data" / "try2_withCC/abundance_C_1880.csv"
output_path = project_root / "data" / "try2_withCC/abundance_C_1880.pkl"
config_path = project_root / "MiCoGPT" / "resources" / "mgm_config.ini"
tokenizer_path = project_root / "MiCoGPT" / "resources" / "mgm_MicroTokenizer.pkl"

print("[path] input_path    =", input_path)
print("[path] output_path   =", output_path)
print("[path] config_path   =", config_path)
print("[path] tokenizer_path=", tokenizer_path)

corpus = construct_simple(
    input_path=str(input_path),
    output_path=str(output_path),
    config_path=str(config_path),
    tokenizer_path=str(tokenizer_path),
    key="genus",
    no_normalize=False,
)

[path] input_path    = /Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/abundance_C_1880.csv
[path] output_path   = /Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/abundance_C_1880.pkl
[path] config_path   = /Users/kiancai/STA24/CWD/STAi/MiCoGPT/MiCoGPT/resources/mgm_config.ini
[path] tokenizer_path= /Users/kiancai/STA24/CWD/STAi/MiCoGPT/MiCoGPT/resources/mgm_MicroTokenizer.pkl
[construct] 从配置文件中读取到 max_len = 512
[construct] tokenizer 加载完成，词表大小 = 9669
Your data will be normalized with the phylogeny mean and std. If you wish to use your own normalization, please set no_normalize=True.
0 samples are dropped for all zeroes


100%|██████████| 1880/1880 [00:00<00:00, 6819.99it/s]


Total 1880 samples.
            Max length is 470.
            Average length is 59.44042553191489.
            Min length is 7.
[construct] MicroCorpus 已构建完成。
[construct] 语料库样本数量 = 1880
[construct] 语料库已保存到：/Users/kiancai/STA24/CWD/STAi/ResMicroDb/data/try2_withCC/abundance_C_1880.pkl


## 3. 检查生成的结果结构

In [4]:
print("corpus 类型:", type(corpus))

# 1. 语料库大小
try:
    num_samples = len(corpus)
    print("样本数（len(corpus)）:", num_samples)
except Exception as e:
    print("无法对 corpus 调用 len()，错误:", e)
    num_samples = 0

# 2. 看看第一个样本长什么样
if num_samples > 0:
    sample = corpus[0]
    print("\n第一个样本类型:", type(sample))

    if isinstance(sample, dict):
        print("样本包含的键:", sample.keys())
        # 尝试看一下 input_ids / attention_mask 的形状和前几个值
        input_ids = sample.get("input_ids", None)
        attention_mask = sample.get("attention_mask", None)

        if input_ids is not None:
            print("input_ids 形状:", getattr(input_ids, "shape", "无 shape 属性"))
            try:
                print("前 20 个 token id:", input_ids[:20].tolist())
            except Exception:
                print("无法直接打印 input_ids[:20].tolist()，可以手动 print(input_ids) 看看")

        if attention_mask is not None:
            print("attention_mask 形状:", getattr(attention_mask, "shape", "无 shape 属性"))
            try:
                print("前 20 个 attention_mask:", attention_mask[:20].tolist())
            except Exception:
                print("无法直接打印 attention_mask[:20].tolist()，可以手动 print(attention_mask) 看看")
    else:
        print("样本不是 dict，直接打印样本内容：")
        print(sample)
else:
    print("corpus 为空，没有样本可以查看。")


corpus 类型: <class 'MiCoGPT.utils.mgm_MicroCorpus.MicroCorpus'>
样本数（len(corpus)）: 1880

第一个样本类型: <class 'dict'>
样本包含的键: dict_keys(['input_ids', 'attention_mask'])
input_ids 形状: torch.Size([512])
前 20 个 token id: [2, 7858, 8081, 6053, 8597, 5318, 5456, 662, 109, 9494, 8050, 6814, 765, 2590, 6748, 228, 1298, 1070, 5464, 280]
attention_mask 形状: torch.Size([512])
前 20 个 attention_mask: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


/Users/kiancai/STA24/CWD/STAi/MiCoGPT/MiCoGPT/utils/mgm_MicroCorpus.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'input_ids': torch.tensor(tokens),


In [5]:
import pickle
from pathlib import Path

pkl_path = Path("/Users/kiancai/STA24/CWD/STAi/MiCoGPT/MiCoGPT/resources/mgm_MicroTokenizer.pkl")  # 根据你的真实路径改一下

with open(pkl_path, "rb") as f:
    from MiCoGPT.utils.mgm_utils import CustomUnpickler
    tokenizer = CustomUnpickler(f).load()

print(type(tokenizer))
print(dir(tokenizer))          # 看看有哪些属性/方法
print(getattr(tokenizer, "vocab", None))  # 看看词表（如果有这个属性）


<class 'MiCoGPT.utils.mgm_MicroCorpus.MicroTokenizer'>
['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_add_tokens', '_additional_special_tokens', '_auto_class', '_batch_encode_plus', '_batch_prepare_for_model', '_bos_token', '_call_one', '_cls_token', '_convert_id_to_token', '_convert_token_to_id', '_convert_token_to_id_with_added_voc', '_create_repo', '_create_trie', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eos_token', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_in_ta